In [874]:
from sqlalchemy import create_engine, text
import pandas as pd
import numpy as np

#CARGARMOS LA EL ARCHIVO E IMPRIMOS INFO PARA VER UN RESUMEN DEL ESTATUS DEL ARCHIVO

In [875]:
path='GSAF5.csv'

In [876]:
shark = pd.read_csv(path, sep=",", encoding = "ISO-8859-1")

In [877]:
shark.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5992 entries, 0 to 5991
Data columns (total 24 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Case Number             5992 non-null   object
 1   Date                    5992 non-null   object
 2   Year                    5992 non-null   int64 
 3   Type                    5992 non-null   object
 4   Country                 5949 non-null   object
 5   Area                    5590 non-null   object
 6   Location                5496 non-null   object
 7   Activity                5465 non-null   object
 8   Name                    5792 non-null   object
 9   Sex                     5425 non-null   object
 10  Age                     3311 non-null   object
 11  Injury                  5965 non-null   object
 12  Fatal (Y/N)             5973 non-null   object
 13  Time                    2779 non-null   object
 14  Species                 3058 non-null   object
 15  Inve

In [878]:
shark.head(2)

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,2016.09.18.c,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.c-NSB.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.c,2016.09.18.c,5993,NaN,NaN
1,2016.09.18.b,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,Chucky Luciano,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.b-Luciano.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.b,2016.09.18.b,5992,NaN,NaN


# REVISAMOS EL TOTAL DE NULOS DE CADA COLUMNA Y ELIMINAMOS LOS QUE TIENE UN PORCENSATAJE MAYOR AL 60%

In [879]:
total_nulos = shark.isnull().sum()
total_nulos[total_nulos > 0].sort_values()
porcentaje_nulos = total_nulos[total_nulos > 0].sort_values()/shark.shape[0]
porcentaje_nulos

href formula              0.000167
href                      0.000501
Investigator or Source    0.002503
Fatal (Y/N)               0.003171
Injury                    0.004506
Country                   0.007176
Name                      0.033378
Area                      0.067089
Location                  0.082777
Activity                  0.087951
Sex                       0.094626
Age                       0.447430
Species                   0.489653
Time                      0.536215
Unnamed: 23               0.999666
Unnamed: 22               0.999833
dtype: float64

In [880]:
columnas_malas = porcentaje_nulos[porcentaje_nulos>0.6].index #REVISAMOS CUALES ASIENDEN A MAS DEL 60 DE NULOS Y LOS GUDARAMOS COMO LISTA
columnas_malas

Index(['Unnamed: 23', 'Unnamed: 22'], dtype='object')

In [881]:
shark.drop(columnas_malas, axis = 1, inplace = True) #ELIMINAREMOS LAS COMUNAS QUE TENGAS UN PORCENTAJE MAYOR A 60% DE VALORES NULLOS

# ANTES DE CONTINUAR, VAMOS A LIMPIAR LOS NOMBRES DE LAS COLUMNAS, PARA PODER TRABAJAR CON ELLAS 

In [882]:
shark.columns

Index(['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Name', 'Sex ', 'Age', 'Injury', 'Fatal (Y/N)', 'Time',
       'Species ', 'Investigator or Source', 'pdf', 'href formula', 'href',
       'Case Number.1', 'Case Number.2', 'original order'],
      dtype='object')

In [883]:
lista_columnas = shark.columns

In [884]:
lista_columnas= lista_columnas.str.strip(' ').str.replace(' ','_').str.replace('.','_') #ESTAMOS QUINTANDO LOS ESPACIOS EXTERNOS , Y LOS INTERNOS LOS CCAMBIAMOS POR _ AL IGUAL QUE LOS PUNTOS '.'
lista_columnas = list(lista_columnas)

C:\Users\sistemas\AppData\Local\Temp\ipykernel_1620\3236888886.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  lista_columnas= lista_columnas.str.strip(' ').str.replace(' ','_').str.replace('.','_') #ESTAMOS QUINTANDO LOS ESPACIOS EXTERNOS , Y LOS INTERNOS LOS CCAMBIAMOS POR _ AL IGUAL QUE LOS PUNTOS '.'


In [885]:
shark.columns = lista_columnas #CONFIRMAMOS LOS CAMBIOS DE LAS COLUMNAS
shark.columns

Index(['Case_Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Name', 'Sex', 'Age', 'Injury', 'Fatal_(Y/N)', 'Time',
       'Species', 'Investigator_or_Source', 'pdf', 'href_formula', 'href',
       'Case_Number_1', 'Case_Number_2', 'original_order'],
      dtype='object')

# DE LOS PAISES DONDE HAN HABIDO ATAQUES, VOY A ELIMNAR TODOS LOS ATAQUES AISLADOS, Y SOLO CON LOS PAISES QUE TIENE EL MAYOR REGISTRO DE ATAQUES

In [886]:
shark.shape # DIMENSION ACTUAL DE MI DATAFRAME

(5992, 22)

In [887]:
lista_country = shark.Country.value_counts()/len(shark) 
lista_country # LISTA DE PORCENTAHE DE ATAQUES POR PAIS

USA                           0.353138
AUSTRALIA                     0.213451
SOUTH AFRICA                  0.094292
PAPUA NEW GUINEA              0.022196
NEW ZEALAND                   0.020861
                                ...   
NORTH ATLANTIC OCEAN          0.000167
MAYOTTE                       0.000167
GABON                         0.000167
ANDAMAN / NICOBAR ISLANDAS    0.000167
CEYLON (SRI LANKA)            0.000167
Name: Country, Length: 203, dtype: float64

In [888]:
columnas_malas_2 = lista_country[lista_country >0.02] # ME QUEDARE CON LOS PASES CON UN REGISTRO MAYOR DEL 2% O EN ESTE CASO CON MAS DE UN ATAQUE RESGITRADO
columnas_malas_2

USA                 0.353138
AUSTRALIA           0.213451
SOUTH AFRICA        0.094292
PAPUA NEW GUINEA    0.022196
NEW ZEALAND         0.020861
Name: Country, dtype: float64

In [889]:
lista_cuntry_malos = list(shark[(shark.Country != 'USA')&(shark.Country != 'AUSTRALIA')&(shark.Country != 'SOUTH AFRICA')&(shark.Country != 'PAPUA NEW GUINEA')&(shark.Country != 'NEW ZEALAND')]['Country'].index)
len(lista_cuntry_malos) #TODOS LOS QUE NO SON A ESTOS PAISES LOS VOY A GUARDAR EN UNA LISTA 

1774

In [890]:
shark.drop(lista_cuntry_malos, inplace = True) # ELIMINO TODAS LAS FILAS DE LOS PAISES QUE SOLO REGISTRARON UN ATAQUE

In [891]:
shark.shape

(4218, 22)

In [892]:
shark.shape

(4218, 22)

# REALIZAMOS UN ANALISIS DE LOS NULLOS QUE AUN ME APARECEN EN LAS COLUMNAS 

In [893]:
shark.isnull().sum() # ME DOY CUENTA QUE YA SE HA REDUCUD

Case_Number                  0
Date                         0
Year                         0
Type                         0
Country                      0
Area                        31
Location                   109
Activity                   320
Name                        66
Sex                        340
Age                       1645
Injury                      14
Fatal_(Y/N)                  4
Time                      1951
Species                   1843
Investigator_or_Source      10
pdf                          0
href_formula                 1
href                         3
Case_Number_1                0
Case_Number_2                0
original_order               0
dtype: int64

# VAMOS A UNIFICAR LAS COLUMNAS Y RELLENAR LOS NULLOS DE SEX

In [894]:
shark.Sex.value_counts()

M      3442
F       431
M         2
lli       1
N         1
.         1
Name: Sex, dtype: int64

In [895]:
shark.Sex.fillna('DESCONOCIDO', inplace = True)

In [896]:
shark.Sex.value_counts()

M              3442
F               431
DESCONOCIDO     340
M                 2
lli               1
N                 1
.                 1
Name: Sex, dtype: int64

In [897]:
shark_2 = shark.copy()

In [898]:
shark_2['Sex'] = np.where(shark_2.Sex.str.startswith('M'), 'MACHO',shark_2.Sex )

In [899]:
shark_2['Sex'] = np.where(shark_2.Sex.str.startswith('F'), 'HEMBRA',shark_2.Sex )

In [900]:
shark_2['Sex'] = np.where(shark_2.Sex.str.startswith(('MACHO','HEMBRA','DESCONOCIDO')), shark_2.Sex , 'DESCONOCIDO')

In [901]:
shark_2.Sex.value_counts()

MACHO          3444
HEMBRA          431
DESCONOCIDO     343
Name: Sex, dtype: int64

In [902]:
shark = shark_2  #GUADARMOS EL FATAFRAME AL ORIGINAL

# REVISAREMOS ACTIVITY

In [903]:
shark.isnull().sum() # ME DOY CUENTA QUE YA SE HA REDUCUD

Case_Number                  0
Date                         0
Year                         0
Type                         0
Country                      0
Area                        31
Location                   109
Activity                   320
Name                        66
Sex                          0
Age                       1645
Injury                      14
Fatal_(Y/N)                  4
Time                      1951
Species                   1843
Investigator_or_Source      10
pdf                          0
href_formula                 1
href                         3
Case_Number_1                0
Case_Number_2                0
original_order               0
dtype: int64

In [904]:
shark.Activity.value_counts()

Surfing                                809
Swimming                               552
Fishing                                293
Spearfishing                           199
Wading                                 124
                                      ... 
Free diving for abalone (surfacing)      1
40' fishing boat sank                    1
Fishing for yellowtail                   1
4 men were bathing                       1
Name: Activity, Length: 973, dtype: int64

In [905]:
shark_2=shark.copy()

In [906]:
shark_2.Activity.fillna('No se sabe', inplace = True)

In [907]:
shark_2.Activity = shark_2.Activity.str.lower()

In [908]:
shark_2.Activity.value_counts()

surfing                                                         809
swimming                                                        552
no se sabe                                                      320
fishing                                                         293
spearfishing                                                    199
                                                               ... 
thrashing the water / imitating the shark victim from "jaws"      1
fishing for white shark                                           1
skindiving,                                                       1
surfing, fell off surfboard                                       1
4 men were bathing                                                1
Name: Activity, Length: 955, dtype: int64

In [909]:
def cambiar_Activity(Activity): 
    if Activity == 'surfing':
        return Activity
    if Activity == 'swimming':
        return Activity
    if Activity == 'fishing':
        return Activity
    if Activity == 'wading':
        return Activity
    if 'fish' in Activity:
        return 'fishing'
    if 'surf' in Activity:
        return 'surfing'
    if 'swim' in Activity:
        return 'swimming'
    if 'wad' in Activity:
        return 'wading'
    

    if 'boarding' in Activity:
        return 'body boarding'
    
    if 'snork' in Activity:
        return 'snorkeling'
    
    if 'diving' in Activity:
        return 'diving'
    

    if 'bathing' in Activity:
        return 'bathing'
    
    if 'kayak' in Activity:
        return 'kayaking'
    
    if 'bathing' in Activity or 'floating' in Activity:
        return 'floating'
    
    
    

    #if '4-Wheel' in Activity or 'All-Wheel' in Activity:
    #    return 'All-Wheel Drive'
    return 'otras actividades'

In [910]:
shark_2['Activity'] = shark_2.Activity.apply(cambiar_Activity)

In [911]:
shark_2.Activity.value_counts()

surfing              1032
otras actividades     911
fishing               804
swimming              694
diving                304
wading                135
body boarding         115
bathing               109
snorkeling             51
floating               34
kayaking               29
Name: Activity, dtype: int64

In [912]:
shark = shark_2

# REVISAREMOS Fatal_(Y/N)

In [913]:
shark.isnull().sum() # ME DOY CUENTA QUE YA SE HA REDUCUD

Case_Number                  0
Date                         0
Year                         0
Type                         0
Country                      0
Area                        31
Location                   109
Activity                     0
Name                        66
Sex                          0
Age                       1645
Injury                      14
Fatal_(Y/N)                  4
Time                      1951
Species                   1843
Investigator_or_Source      10
pdf                          0
href_formula                 1
href                         3
Case_Number_1                0
Case_Number_2                0
original_order               0
dtype: int64

In [914]:
shark['Fatal_(Y/N)'].value_counts()

N          3334
Y           808
UNKNOWN      62
 N            7
N             1
#VALUE!       1
n             1
Name: Fatal_(Y/N), dtype: int64

In [915]:
shark_2 = shark.copy()

In [916]:
shark_2['Fatal_(Y/N)'].fillna('UNKNOWN', inplace = True)

In [917]:
def cambiar_Fatal(Activity): 
    if Activity == 'N':
        return Activity
    if Activity == 'Y':
        return Activity
    if Activity == 'UNKNOWN':
        return Activity
    
    if 'N' in Activity or 'n' in Activity:
        return 'N'
    
    
    

    #if '4-Wheel' in Activity or 'All-Wheel' in Activity:
    #    return 'All-Wheel Drive'
    return 'UNKNOWN'

In [918]:
shark_2['Fatal_(Y/N)'] = shark_2['Fatal_(Y/N)'].apply(cambiar_Fatal)

In [919]:
shark_2['Fatal_(Y/N)'].value_counts() # REDUCIMOS A SOLO N , Y Y UNKNOWN

N          3343
Y           808
UNKNOWN      67
Name: Fatal_(Y/N), dtype: int64

In [920]:
shark = shark_2

# RELLENAREMOS LAS COLUMNAS QUE TIENE POCOS NULL, CON UN STRING O VALOR DEACUERDO A LO QUE SOLICITA

In [852]:
shark.isnull().sum() # ME DOY CUENTA QUE YA SE HA REDUCUD

Case_Number                  0
Date                         0
Year                         0
Type                         0
Country                      0
Area                        31
Location                   109
Activity                     0
Name                        66
Sex                          0
Age                       1645
Injury                      14
Fatal_(Y/N)                  0
Time                      1951
Species                   1843
Investigator_or_Source      10
pdf                          0
href_formula                 1
href                         3
Case_Number_1                0
Case_Number_2                0
original_order               0
dtype: int64

In [923]:
shark_2 = shark.copy()

In [925]:
shark_2.Area.fillna('SIN AREA', inplace = True)

In [926]:
shark_2.Name.fillna('Anonymous', inplace = True)

In [927]:
shark_2.Name.value_counts()

male              306
Anonymous          74
female             61
boat               14
boy                 8
                 ... 
Jesse Spencer       1
Joel A. Borges      1
Janet Ferguson      1
Taylor Warnock      1
Ahmun               1
Name: Name, Length: 3684, dtype: int64

In [928]:
shark_2.Area.value_counts()

Florida                               990
New South Wales                       468
Queensland                            300
Hawaii                                282
California                            276
                                     ... 
North & South Carolina                  1
Florida                                 1
Near Bougainville (North Solomons)      1
Northern District                       1
Between New Ireland & New Britain       1
Name: Area, Length: 112, dtype: int64

In [934]:
shark_2.Location.value_counts()

New Smyrna Beach, Volusia County                  157
Daytona Beach, Volusia County                      28
Myrtle Beach, Horry County                         17
Ponce Inlet, Volusia County                        17
Melbourne Beach, Brevard County                    15
                                                 ... 
Gleneden Beach                                      1
Whale Beach                                         1
Hole-in-the Wall                                    1
Wreck of the Navy Barge, 22 miles SE of Rudee       1
Ocracoke Inlet                                      1
Name: Location, Length: 2795, dtype: int64

In [936]:
shark_2.Location.fillna('Sin ubicacion', inplace = True)

In [938]:
shark_2.isnull().sum() # ME DOY CUENTA QUE YA SE HA REDUCUD

Case_Number                  0
Date                         0
Year                         0
Type                         0
Country                      0
Area                         0
Location                     0
Activity                     0
Name                         0
Sex                          0
Age                       1645
Injury                      14
Fatal_(Y/N)                  0
Time                      1951
Species                   1843
Investigator_or_Source      10
pdf                          0
href_formula                 1
href                         3
Case_Number_1                0
Case_Number_2                0
original_order               0
dtype: int64

In [947]:
shark_2.Injury = shark_2.Injury.str.lower()

In [952]:
shark_2.Injury.fillna('no details', inplace = True)

In [999]:
def cambiar_Injury(Activity): 
    if Activity == 'fatal':
        return Activity
    if Activity == 'survived':
        return Activity
    #if Activity == 'UNKNOWN':
    #    return Activity
    
    #if 'N' in Activity or 'n' in Activity:
     #   return 'N'
    
    if 'survived' in Activity:
        return 'survived'
    if 'bit' in Activity:
        return 'bitten'
    if 'fatal' in Activity:
        return 'fatal'
    if 'injur' in Activity:
        return 'injury'
    if 'lacera' in Activity:
        return 'lacerations'
    if 'kill' in Activity:
        return 'fatal'
    
    if 'severed' in Activity:
        return 'severed'
    if 'puncture' in Activity:
        return 'injury'
    
    if 'abrasion' in Activity:
        return 'lacerations'
    
    #if '4-Wheel' in Activity or 'All-Wheel' in Activity:
    #    return 'All-Wheel Drive'
    return 'Otros'

In [1000]:
shark_2['Injury'] = shark_2['Injury'].apply(cambiar_Injury)

In [1001]:
shark_2.Injury.value_counts()

bitten         1214
lacerations    1058
injury          884
fatal           526
Otros           421
survived         58
severed          57
Name: Injury, dtype: int64

In [1002]:
shark_2.shape

(4218, 22)

In [ ]:

Time                      1951
Species                   1843
Investigator_or_Source      10
pdf                          0
href_formula                 1
href                         3

In [1003]:
shark_2.Investigator_or_Source

0                             Orlando Sentinel, 9/19/2016
1                             Orlando Sentinel, 9/19/2016
2                             Orlando Sentinel, 9/19/2016
3                                      The Age, 9/18/2016
4                                      The Age, 9/16/2016
                              ...                        
5985    NY Sun, 9/9/1906, referring to account by Loui...
5986    L. Becke in New York Sun, 9/9/1906; L. Schultz...
5987                     H. Taunton; N. Bartlett,  p. 234
5988                H. Taunton; N. Bartlett,  pp. 233-234
5989                 F. Schwartz, p.23; C. Creswell, GSAF
Name: Investigator_or_Source, Length: 4218, dtype: object

In [1004]:
shark_2.Investigator_or_Source.fillna('Sin INFO', inplace = True)

In [1006]:
shark_2.Investigator_or_Source.value_counts()

S. Petersohn, GSAF                                                             82
C. Creswell, GSAF                                                              81
R. Collier                                                                     53
T. Peake, GSAF                                                                 48
M. Levine, GSAF                                                                39
                                                                               ..
Deseret News, 8/19/1993                                                         1
G. Balazs; T. Allen, p.117; Hawaii Department of Land and Natural Resources     1
R. Collier, p.174                                                               1
Charlotte Observer, 8/19/1993, p.2C; F. Schwartz, p.23                          1
F. Schwartz, p.23; C. Creswell, GSAF                                            1
Name: Investigator_or_Source, Length: 3449, dtype: int64

# VAMOS RELLENAR Y REVISAR LAS TAVLAS HREF Y HREF_FORMULA

In [1012]:
shark= shark_2

In [1091]:
shark_2 = shark.copy() # GUARDAMOS ANTES DE INCIAR   

In [ ]:
#shark_2.drop_duplicates(subset = ['href_formula','href']).shape # PROBAMOS EL DROP DUPLICATE PERO NOS DAMOS CUENTA QUE AHI LINEAS QUE NO SON IGUALES

In [1092]:
shark_2.href.fillna('Sin_href', inplace = True) # PRIMERO LLENAMOS LOS VACIOS
shark_2.href_formula.fillna('Sin_href', inplace = True)

In [1093]:
shark_2.href = shark_2.href.str.replace(' ','%20') # LOS LINKS ES COLOCAMOS EL %20 PARA LOS ESPACIOS
shark_2.href_formula = shark_2.href_formula.str.replace(' ','%20')

In [1094]:
lista_href = shark_2[shark_2.href != shark_2.href_formula] # OBTENEMOS LAS FILAS QUE NO SON IGUALES EN AMBAS COMLUMNAS

In [1095]:
lista_href['href']

61      http://sharkattackfile.net/spreadsheets/pdf_di...
114     http://sharkattackfile.net/spreadsheets/pdf_di...
134     http://sharkattackfile.net/spreadsheets/pdf_di...
180     http://sharkattackfile.net/spreadsheets/pdf_di...
193                                              Sin_href
232     http://sharkattackfile.net/spreadsheets/pdf_di...
262     http://sharkattackfile.net/spreadsheets/pdf_di...
263     http://sharkattackfile.net/spreadsheets/pdf_di...
264     http://sharkattackfile.net/spreadsheets/pdf_di...
271     http://sharkattackfile.net/spreadsheets/pdf_di...
272     http://sharkattackfile.net/spreadsheets/pdf_di...
293     http://sharkattackfile.net/spreadsheets/pdf_di...
323     http://sharkattackfile.net/spreadsheets/pdf_di...
347     http://sharkattackfile.net/spreadsheets/pdf_di...
362     http://sharkattackfile.net/spreadsheets/pdf_di...
363     http://sharkattackfile.net/spreadsheets/pdf_di...
364     http://sharkattackfile.net/spreadsheets/pdf_di...
377     http:/

In [1097]:
lista_href_3 = list(lista_href['href'].index) # GUARDAMOS LAS FILAS PARA POSTERIOR USAR LAS
lista_href_3

[61,
 114,
 134,
 180,
 193,
 232,
 262,
 263,
 264,
 271,
 272,
 293,
 323,
 347,
 362,
 363,
 364,
 377,
 378,
 380,
 381,
 382,
 383,
 448,
 449,
 823,
 2274,
 2477,
 3019,
 3603,
 4642,
 5317,
 5686]

In [1132]:
shark_3 = shark_2.copy() # GUARDAMOS EN OTRO LINK PARA PRUEBAS

In [1102]:
import requests
from bs4 import BeautifulSoup

# en el programa que viene vamos a cargar los links de la lista de links que difiere, y vamos a realizar un request para ver si el link es funcional o no, vamos a guardar los links funciones en ambatas tablas para que ahora si sean iguales
# pero en el caso de que los links esten caidos, los vamos a guardar con una comentario de link caido, esto lo guardaremos en ambas columnas y para despues solo quedarnos con una de ellas.

In [1133]:
for item in lista_href_3:
    link= shark_3.href_formula[item]
    link_2 = shark_3.href[item]
    try:
        response_link = requests.get(link) # REQUEST DE LA PAGINA
        if str(response_link) == '<Response [200]>':
            #print('href_formula    link bueno',link)
            shark_3.href = link
        else:
            response_link_2 = requests.get(link_2) # REQUEST DE LA PAGINA
            if str(response_link_2) == '<Response [200]>':
                #print('href    link bueno',link_2)
                shark_3.href_formula = link_2
    except:
        try:
            response_link_2 = requests.get(link_2) # REQUEST DE LA PAGINA
            if str(response_link_2) == '<Response [200]>':
                shark_3.href_formula = link_2
                #print('href    link bueno',link_2)
        except:
            if 'http' in link:
                #print('links malos ',link)
                shark_3.href = 'LINK_CAIDO____'+link
            if 'http' in link_2:
                #print('links malos ',link_2)
                shark_3.href_formula = 'LINK_CAIDO____'+link_2

In [1134]:
shark_3.href[61] # VERIFICAMOS UNA LINA PARA REVISAR QUE YA SON IGUALES

'http://sharkattackfile.net/spreadsheets/pdf_directory/2016.05.21.a-Girl.pdf'

In [1135]:
shark_3.href_formula[61] # VERIFICAMOS UNA LINA PARA REVISAR QUE YA SON IGUALES

'http://sharkattackfile.net/spreadsheets/pdf_directory/2016.05.21.a-Girl.pdf'

In [1140]:
shark_3.shape # REVISAMOS LA DIMENCION ANTES DE DROPEAR

(4218, 22)

In [1143]:
shark_3 = shark_3.T.drop_duplicates().T # se elimina una columan duplicada 

In [1145]:
shark_3.shape

(4218, 21)

In [1146]:
shark = shark_3

In [1151]:
shark.isnull().sum() # ME DOY CUENTA QUE YA SE HA REDUCUD

Case_Number                  0
Date                         0
Year                         0
Type                         0
Country                      0
Area                         0
Location                     0
Activity                     0
Name                         0
Sex                          0
Age                       1645
Injury                       0
Fatal_(Y/N)                  0
Time                      1951
Species                   1843
Investigator_or_Source       0
pdf                          0
href_formula                 0
Case_Number_1                0
Case_Number_2                0
original_order               0
dtype: int64

In [1177]:
lista_time = list(shark.Time.value_counts().index)


# como aun tenemos varios nullos, se les realizara un firltro , en donde eliminaremos las filas donde AGE, TIME Y SPECIES TENGAN NULL

In [1264]:
shark_3 = shark_2.drop_duplicates(subset = ['Age', 'Time', 'Species'], keep = 'last')

In [1265]:
shark_3.shape # se realiza un shape para ver que dimension nos queda la tabla

(2867, 21)

In [1260]:
shark_3 = shark

In [1261]:
len(shark)

4218

In [1266]:
shark_3.Age.value_counts()

19         95
18         92
17         90
15         87
16         85
           ..
Elderly     1
mid-20s     1
Ca. 33      1
            1
young       1
Name: Age, Length: 125, dtype: int64

In [1267]:
shark_3.Time.value_counts()

Afternoon    101
11h00         86
14h00         72
12h00         71
Morning       66
            ... 
16h14          1
17h11          1
00h30          1
14h37          1
01h50          1
Name: Time, Length: 316, dtype: int64

In [1268]:
shark_3.Age.fillna('0', inplace = True)

C:\Users\sistemas\AppData\Local\Temp\ipykernel_1620\697857519.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shark_3.Age.fillna('0', inplace = True)


# la tabla de edad, se rebisara dato por dato, en donde se pueda guardar como entero, en donde no se guardara como 0.

In [1269]:
for item in list(shark_3.index):
    try:
        edad = int(shark_3.Age[item])
        shark_3.Age[item] = edad
    except:
        shark_3.Age[item] = 0

C:\Users\sistemas\AppData\Local\Temp\ipykernel_1620\1692768396.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shark_3.Age[item] = edad
C:\Users\sistemas\AppData\Local\Temp\ipykernel_1620\1692768396.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shark_3.Age[item] = 0


In [1270]:
shark_3.Age.value_counts()

0     704
19     95
18     92
17     90
15     87
     ... 
84      1
73      1
87      1
67      1
78      1
Name: Age, Length: 77, dtype: int64

In [1272]:
shark_3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2867 entries, 1 to 5989
Data columns (total 21 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Case_Number             2867 non-null   object
 1   Date                    2867 non-null   object
 2   Year                    2867 non-null   object
 3   Type                    2867 non-null   object
 4   Country                 2867 non-null   object
 5   Area                    2867 non-null   object
 6   Location                2867 non-null   object
 7   Activity                2867 non-null   object
 8   Name                    2867 non-null   object
 9   Sex                     2867 non-null   object
 10  Age                     2867 non-null   object
 11  Injury                  2867 non-null   object
 12  Fatal_(Y/N)             2867 non-null   object
 13  Time                    2072 non-null   object
 14  Species                 2204 non-null   object
 15  Inve

In [1273]:
shark_3.isnull().sum() # ME DOY CUENTA QUE YA SE HA REDUCUD

Case_Number                 0
Date                        0
Year                        0
Type                        0
Country                     0
Area                        0
Location                    0
Activity                    0
Name                        0
Sex                         0
Age                         0
Injury                      0
Fatal_(Y/N)                 0
Time                      795
Species                   663
Investigator_or_Source      0
pdf                         0
href_formula                0
Case_Number_1               0
Case_Number_2               0
original_order              0
dtype: int64

# vamos a remplazar las horarios de 11hr00 a 11:00:00, para el caso de querer cambiarlo a time, sin embargo no seria conveniente debido a que solo cuenta con hora

In [1274]:
shark_3.Time.fillna('00:00:00', inplace = True)

C:\Users\sistemas\AppData\Local\Temp\ipykernel_1620\450696793.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shark_3.Time.fillna('00:00:00', inplace = True)


In [1275]:
shark_3.Time.value_counts()

00:00:00     795
Afternoon    101
11h00         86
14h00         72
12h00         71
            ... 
13h345         1
               1
15h19          1
06h47          1
01h50          1
Name: Time, Length: 317, dtype: int64

In [1276]:
shark_2 = shark_3.copy()

# aqui realizo el cambio de horas a formato 00:00:00 , los casos que no se adapte este se guardara como 00:00:00

In [1277]:
for item in list(shark_3.index):
    time = shark_3.Time[item]
    try:
        texto2 = re.search("([0-9][0-9])(h)([0-9][0-9])", time)
        shark_3.Time[item] = texto2.group(1)+':'+texto2.group(3)+':00'
    except:
        shark_3.Time[item] ='00:00:00'

C:\Users\sistemas\AppData\Local\Temp\ipykernel_1620\1789207465.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shark_3.Time[item] = texto2.group(1)+':'+texto2.group(3)+':00'
C:\Users\sistemas\AppData\Local\Temp\ipykernel_1620\1789207465.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shark_3.Time[item] ='00:00:00'


In [1278]:
shark_3.Time

1       11:00:00
2       10:43:00
6       15:15:00
7       14:30:00
9       00:00:00
          ...   
5935    00:00:00
5984    00:00:00
5985    00:00:00
5986    00:00:00
5989    00:00:00
Name: Time, Length: 2867, dtype: object

In [1281]:
shark = shark_3

In [1283]:
shark_2 = shark.copy()

# En el caso de spacies , se agregara como sin identificar en la tabla para los casos NULL

In [1284]:
shark_2.Species.fillna('SIN_IDENTIFICAR', inplace = True)

In [1285]:
shark_2.Species

1                                         SIN_IDENTIFICAR
2                                         SIN_IDENTIFICAR
6                                          3' to 4' shark
7                                        Tiger shark, 10?
9                                         SIN_IDENTIFICAR
                              ...                        
5935                               White shark, 1,900-lb 
5984                                         Blue pointer
5985                                        Blue pointers
5986    Said to involve a grey nurse shark that leapt ...
5989                                      SIN_IDENTIFICAR
Name: Species, Length: 2867, dtype: object

In [1286]:
shark= shark_2

# VAMOS A LIMPIAR LAS TABLAS CASE_NUMBER PARA SOLO QUEDARNOS CON UNA

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2867 entries, 1 to 5989
Data columns (total 21 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Case_Number             2867 non-null   object
 1   Date                    2867 non-null   object
 2   Year                    2867 non-null   object
 3   Type                    2867 non-null   object
 4   Country                 2867 non-null   object
 5   Area                    2867 non-null   object
 6   Location                2867 non-null   object
 7   Activity                2867 non-null   object
 8   Name                    2867 non-null   object
 9   Sex                     2867 non-null   object
 10  Age                     2867 non-null   object
 11  Injury                  2867 non-null   object
 12  Fatal_(Y/N)             2867 non-null   object
 13  Time                    2867 non-null   object
 14  Species                 2867 non-null   object
 15  Inve

In [1293]:
shark.head(2)

,Case_Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Injury,Fatal_(Y/N),Time,Species,Investigator_or_Source,pdf,href_formula,Case_Number_1,Case_Number_2,original_order
1,2016.09.18.b,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",surfing,Chucky Luciano,MACHO,...,lacerations,N,11:00:00,SIN_IDENTIFICAR,"Orlando Sentinel, 9/19/2016",2016.09.18.b-Luciano.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.b,2016.09.18.b,5992
2,2016.09.18.a,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",surfing,male,MACHO,...,lacerations,N,10:43:00,SIN_IDENTIFICAR,"Orlando Sentinel, 9/19/2016",2016.09.18.a-NSB.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.a,2016.09.18.a,5991


In [1356]:
shark_2 = shark.copy()

In [1357]:
lista_case = shark_2[shark_2.Case_Number_1 != shark_2.Case_Number_2] # OBTENEMOS LAS FILAS QUE NO SON IGUALES EN AMBAS COMLUMNAS

In [1358]:
lista_case[['Case_Number','Case_Number_1','Case_Number_2']]

,Case_Number,Case_Number_1,Case_Number_2
97,2016.01.24.b,2015.01.24.b,2016.01.24.b
116,2015.12.23,2015.11.07,2015.12.23
121,2015.10.28.a,2015.10.28,2015.10.28.a


In [1359]:
lista_case_2 = shark_2[shark_2.Case_Number != shark_2.Case_Number_1] # OBTENEMOS LAS FILAS QUE NO SON IGUALES EN AMBAS COMLUMNAS

In [1360]:
lista_case_2[['Case_Number','Case_Number_1','Case_Number_2']]

,Case_Number,Case_Number_1,Case_Number_2
97,2016.01.24.b,2015.01.24.b,2016.01.24.b
116,2015.12.23,2015.11.07,2015.12.23
121,2015.10.28.a,2015.10.28,2015.10.28.a
169,2015.07-10,2015.07.10,2015.07.10


In [1363]:
shark_2.Case_Number[97] = shark_2.Case_Number_1[97] = shark_2.Case_Number_2[97] = '2016.01.24.b'
shark_2.Case_Number[116] = shark_2.Case_Number_1[116] = shark_2.Case_Number_2[116] = '2015.12.23'
shark_2.Case_Number[121] = shark_2.Case_Number_1[121] = shark_2.Case_Number_2[121] = '2015.10.28.a'
shark_2.Case_Number[169] = shark_2.Case_Number_1[169] = shark_2.Case_Number_2[169] = '2015.07.10'

In [1369]:
shark_2.shape

(2867, 21)

In [1367]:
shark_2['Case_Number_1'][169]

'2015.07.10'

In [1368]:
shark_2['Case_Number_2'][169]

'2015.07.10'

In [1370]:
shark_2 = shark_2.T.drop_duplicates().T # se elimina una columan duplicada 

In [1372]:
shark_2.shape

(2867, 19)

In [1376]:
shark_2.head(2)

,Case_Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal_(Y/N),Time,Species,Investigator_or_Source,pdf,href_formula,original_order
1,2016.09.18.b,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",surfing,Chucky Luciano,MACHO,36,lacerations,N,11:00:00,SIN_IDENTIFICAR,"Orlando Sentinel, 9/19/2016",2016.09.18.b-Luciano.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,5992
2,2016.09.18.a,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",surfing,male,MACHO,43,lacerations,N,10:43:00,SIN_IDENTIFICAR,"Orlando Sentinel, 9/19/2016",2016.09.18.a-NSB.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,5991


In [1377]:
shark = shark_2

# guardamos la tabla limpia como cvs

In [1378]:
shark.to_csv('tabla_SHARK_limpio.cvs', index=False) #GUARDAMOS EN UNA TABLA CVS

In [1379]:
shark.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2867 entries, 1 to 5989
Data columns (total 19 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Case_Number             2867 non-null   object
 1   Date                    2867 non-null   object
 2   Year                    2867 non-null   object
 3   Type                    2867 non-null   object
 4   Country                 2867 non-null   object
 5   Area                    2867 non-null   object
 6   Location                2867 non-null   object
 7   Activity                2867 non-null   object
 8   Name                    2867 non-null   object
 9   Sex                     2867 non-null   object
 10  Age                     2867 non-null   object
 11  Injury                  2867 non-null   object
 12  Fatal_(Y/N)             2867 non-null   object
 13  Time                    2867 non-null   object
 14  Species                 2867 non-null   object
 15  Inve

In [1380]:
shark.isnull().sum() # ME DOY CUENTA QUE YA SE HA REDUCUD

Case_Number               0
Date                      0
Year                      0
Type                      0
Country                   0
Area                      0
Location                  0
Activity                  0
Name                      0
Sex                       0
Age                       0
Injury                    0
Fatal_(Y/N)               0
Time                      0
Species                   0
Investigator_or_Source    0
pdf                       0
href_formula              0
original_order            0
dtype: int64